# OpenStreet of Map Melbourne, AUS Case Study¶

This data set is downloaded from a ready extract of more than 200 areas world wide on BBBBike.
    - https://download.bbbike.org/osm/

### Understanding the map XML file and parrsing it using xml library in Python

#1
Use the iterative parsing to process the map file and find out what tags are there, to get the feeling on how much of which data expected to have in the map.

In [2]:
import xml.etree.cElementTree as ET
import pprint
from collections import defaultdict

"""
Create a function called count_tags which iterate through the xml file looking for tags
and count those tags.
"""
def count_tags(filename):
    tags = defaultdict(int)
    tree = ET.parse('Melbourne.osm')
    root = tree.getroot()
    for element in root.iter():
        tags[element.tag] +=1
    return tags

def test():
    tags = count_tags('Melbourne.osm')
    pprint.pprint(tags)

if __name__ == "__main__":
    test()

defaultdict(<class 'int'>,
            {'bounds': 1,
             'member': 116983,
             'nd': 2708537,
             'node': 2301716,
             'osm': 1,
             'relation': 11990,
             'tag': 2313691,
             'way': 356310})


#### The next is to explore the data a bit more. Before processing the data and add it into database, check the "k" value for each "<tag>" and see if there are any potential problems. I would like to change the data model and expand the "addr:street" type of keys to a dictionary like this:
{"address": {"street": "Some value"}}
So, I have to see if we have such tags, and if we have any tags with problematic characters.

Counting of each of four tag categories in a dictionary:
  "lower", for tags that contain only lowercase letters and are valid,
  "lower_colon", for otherwise valid tags with a colon in their names,
  "problemchars", for tags with problematic characters, and
  "other", for other tags that do not fall into the other three categories.

In [3]:
import re

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

def key_type(element, keys):
    if element.tag == "tag":   
        # for tag in element.iter():
        if lower.match(element.attrib['k']):
            keys['lower'] += 1
        elif lower_colon.match(element.attrib['k']):
            keys['lower_colon'] += 1
        elif problemchars.match(element.attrib['k']):
            keys['problemchars'] += 1
        else:
            keys['other'] += 1
    return keys


def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys

def test():
    keys = process_map('Melbourne.osm')
    pprint.pprint(keys)

if __name__ == "__main__":
    test()

{'lower': 1635015, 'lower_colon': 676293, 'other': 2383, 'problemchars': 0}


### Auditing the street name

The list of street names are listed in 'expected', 'audit_street_type'would list all the street names if not included. 

In [4]:

osmfile = "Melbourne.osm"

street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
stret_types = defaultdict(set)


#expected street types
expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons", "Circle", "Crescent", "Gate", "Terrace", "Grove", "Way"]

#
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)
   
#
def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


#
def audit_street(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

audit_street_results = audit_street('Melbourne.osm')

def test():
    pprint.pprint(audit_street_results)

if __name__ == "__main__":
    test()

defaultdict(<class 'set'>,
            {'251': {'251'},
             'Airport': {'Arrival Drive Melbourne Airport'},
             'Alley': {'Tin Alley', 'Croft Alley', 'Globe Alley'},
             'Allister': {'Allister'},
             'Arcade': {'Campbell Arcade',
                        'Kings Arcade',
                        'Preston Chambers Arcade'},
             'Ave': {'Suite 1, St Vincent’s Private Hospital, 5 Studley Ave',
                     'Victoria Ave'},
             'Avp': {'Bayside Avp'},
             'Balwyn': {'Balwyn'},
             'Blvd': {'Quay Blvd'},
             'Boardwalk': {'The Boardwalk'},
             'Bouf': {'Logis Bouf'},
             'Bourke': {'Little Bourke', 'Bourke'},
             'Briars': {'The Briars'},
             'Brighton': {'Brighton'},
             'Broadway': {'Broadway'},
             'Carlisle': {'Carlisle'},
             'Carr': {'Carr'},
             'Causeway': {'The Causeway'},
             'Centerway': {'The Centerway', 'Centerway

It can be found that there are many discrepancies in the street names such as:

            "St.": "Street",
            "St":"Street",
            "Dr.": "Drive",
            "Dr,": "Drive",
            "Rd": "Road",
            "Ro": "Road",
            "rd":"Road",

### Correcting the street names

The function update_name will update the upon wrong names to correct ones.

In [5]:
osmfile = "Melbourne.osm"
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

mapping = { 
            "St.": "Street",
            "St":"Street",
            "Dr.": "Drive",
            "Dr,": "Drive",
            "Rd": "Road",
            "Ro": "Road",
            "rd":"Road",
            }

def update_name(name, mapping, regex):
    m = regex.search(name)
    if m:
        street_type = m.group()
        if street_type in mapping:
            name = re.sub(regex, mapping[street_type], name)

    return name

for street_type, ways in audit_street_results.items():
    for name in ways:
        better_name = update_name(name, mapping, street_type_re)
        print (name, "=>", better_name)

Eildon Parade => Eildon Parade
Ellingworth Parade => Ellingworth Parade
Campbell Parade => Campbell Parade
Thaxted Parade => Thaxted Parade
Marine Parade => Marine Parade
Rowena Parade => Rowena Parade
Robbs Parade => Robbs Parade
Livingstone Parade => Livingstone Parade
Lawson Parade => Lawson Parade
Beaconsfield Parade => Beaconsfield Parade
Gardiner Parade => Gardiner Parade
Bundora Parade => Bundora Parade
Rostrevor Parade => Rostrevor Parade
Robern Parade => Robern Parade
Montana Parade => Montana Parade
Alexandra Parade => Alexandra Parade
Cameron Parade => Cameron Parade
Auburn Parade => Auburn Parade
Kemilworth Parade => Kemilworth Parade
Wildwood Parade => Wildwood Parade
Civic Parade => Civic Parade
Champion Parade => Champion Parade
Prosper Parade => Prosper Parade
Alexandria Parade => Alexandria Parade
Cullis Parade => Cullis Parade
South Parade => South Parade
Polo Parade => Polo Parade
Queens Parade => Queens Parade
Anderson Parade => Anderson Parade
Viggers Parade => Vig

### Auditing the  Zipcode

In [6]:
osmfile = "Melbourne.osm"
zip_type_re = re.compile(r'\d{4}$')

def audit_ziptype(zip_types, zipcode):
    if zipcode[0:1] != 3:
        zip_types[zipcode[0:1]].add(zipcode)
        
def is_zipcode(elem):
    return (elem.attrib['k'] == "addr:postcode")

def audit_zip(osmfile):
    osm_file = open(osmfile, "r")
    zip_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_zipcode(tag):
                    audit_ziptype(zip_types,tag.attrib['v'])
    osm_file.close()
    return zip_types

zip_print = audit_zip(osmfile)

def test():    
    pprint.pprint(dict(zip_print))

if __name__ == '__main__':
    test()

{'2': {'2004', '2181'},
 '3': {'3000',
       '3001',
       '3002',
       '3003',
       '3004',
       '3005',
       '3006',
       '3006;3130',
       '3008',
       '3010',
       '3011',
       '3012',
       '3013',
       '3013s',
       '3015',
       '3016',
       '3018',
       '3019',
       '3020',
       '3021',
       '3022',
       '3023',
       '3025',
       '3026',
       '3027',
       '3028',
       '3029',
       '3030',
       '3031',
       '3031-',
       '3032',
       '3033',
       '3034',
       '3036',
       '3037',
       '3038',
       '3039',
       '3040',
       '3040\u200e',
       '3041',
       '3042',
       '3043',
       '3044',
       '3045',
       '3046',
       '3047',
       '3048',
       '3049',
       '3050',
       '3051',
       '3052',
       '3053',
       '3054',
       '3055',
       '3056',
       '3057',
       '3058',
       '3059',
       '3060',
       '3061',
       '3062',
       '3063',
       '3064',
       '3065',
   

### Correcting the zipcodes

In [7]:

'''
This code will update non 4-digit zipcode.
    - If 8 digit as 8 digits 3006;3130, only the first 4 digits are kept.
    - If 4 digit and ends alphabets such as 3206Unset, only the 4 digits are kept.
    - If 4 number and starts alphabets and space such as VIC 3000, only the 4 digits are kept.
'''
def update_zipcode(zipcode):
    """Clean postcode to a uniform format of 4 digit; Return updated postcode"""
    if re.findall(r'^\d{4}$', zipcode): # 4 digits 3000
        valid_zipcode = zipcode
        return valid_zipcode
    elif re.findall(r'(^\d{4});\d{4}$', zipcode): # 8 digits 3006;3130
        valid_zipcode = re.findall(r'(^\d{4});\d{4}$', zipcode)[0]
        return valid_zipcode  
    elif re.findall(r'(^\d{4})\D*', zipcode): # four number with alphabets such as 3206Unset
        valid_zipcode = re.findall(r'\d{4}', zipcode)[0]
        return valid_zipcode
    elif re.findall(r'(^\D*)s*\d{4}', zipcode): # four number with alphabets and space such as VIC 3000
        valid_zipcode = re.findall(r'\d{4}', zipcode)[0]
        return valid_zipcode   
    else: #return default zipcode to avoid overwriting
        return zipcode   

def test_zip():
    for zips, ways in zip_print.items():
        for name in ways:
            better_name = update_zipcode(name)
            print (name, "=>", better_name)

if __name__ == '__main__':
    test_zip()

3191 => 3191
3163 => 3163
3133 => 3133
3177 => 3177
3101 => 3101
3022 => 3022
3122 => 3122
3429 => 3429
3099 => 3099
3006 => 3006
3804 => 3804
3805 => 3805
3047 => 3047
3103 => 3103
3136 => 3136
3038 => 3038
3013 => 3013
3107 => 3107
3006;3130 => 3006
3028 => 3028
3088 => 3088
3044 => 3044
3143 => 3143
3197 => 3197
3055 => 3055
3048 => 3048
3206 => 3206
3106 => 3106
3025 => 3025
3202 => 3202
3108 => 3108
3104 => 3104
3040 => 3040
3036 => 3036
3167 => 3167
3023 => 3023
3817 => 3817
3206Unset => 3206
3148 => 3148
3008 => 3008
3076 => 3076
3045 => 3045
3053 => 3053
3204 => 3204
3043 => 3043
3020 => 3020
3186 => 3186
3015 => 3015
3427 => 3427
3051 => 3051
3074 => 3074
3124 => 3124
3802 => 3802
3114 => 3114
3050 => 3050
3054 => 3054
3198 => 3198
3183 => 3183
3752 => 3752
3010 => 3010
3132 => 3132
3149 => 3149
3195 => 3195
3097 => 3097
3123 => 3123
3093 => 3093
3113 => 3113
3152 => 3152
3168 => 3168
3073 => 3073
3063 => 3063
3042 => 3042
3078 => 3078
3058 => 3058
3162 => 3162
3128 => 3128
37

### Convert Database into .csv

In [ ]:
import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET

import cerberus

import schema

schema = {
    'node': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'lat': {'required': True, 'type': 'float', 'coerce': float},
            'lon': {'required': True, 'type': 'float', 'coerce': float},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'node_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    },
    'way': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'way_nodes': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'node_id': {'required': True, 'type': 'integer', 'coerce': int},
                'position': {'required': True, 'type': 'integer', 'coerce': int}
            }
        }
    },
    'way_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    }
}

OSM_PATH = "Melbourne.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']  
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  
    #YOUR CODE HERE
    if element.tag == 'node':
        for attrib in element.attrib:
            if attrib in NODE_FIELDS:
                node_attribs[attrib] = element.attrib[attrib]
        
        for child in element:
            node_tag = {}
            if LOWER_COLON.match(child.attrib['k']):
                node_tag['type'] = child.attrib['k'].split(':',1)[0]
                node_tag['key'] = child.attrib['k'].split(':',1)[1]
                node_tag['id'] = element.attrib['id']
                node_tag['value'] = child.attrib['v']
                tags.append(node_tag)
            elif PROBLEMCHARS.match(child.attrib['k']):
                continue
            else:
                node_tag['type'] = 'regular'
                node_tag['key'] = child.attrib['k']
                node_tag['id'] = element.attrib['id']
                node_tag['value'] = child.attrib['v']
                tags.append(node_tag)
        
        return {'node': node_attribs, 'node_tags': tags}
        
    elif element.tag == 'way':
        for attrib in element.attrib:
            if attrib in WAY_FIELDS:
                way_attribs[attrib] = element.attrib[attrib]
        
        position = 0
        for child in element:
            way_tag = {}
            way_node = {}
            
            if child.tag == 'tag':
                if LOWER_COLON.match(child.attrib['k']):
                    way_tag['type'] = child.attrib['k'].split(':',1)[0]
                    way_tag['key'] = child.attrib['k'].split(':',1)[1]
                    way_tag['id'] = element.attrib['id']
                    way_tag['value'] = child.attrib['v']
                    tags.append(way_tag)
                elif PROBLEMCHARS.match(child.attrib['k']):
                    continue
                else:
                    way_tag['type'] = 'regular'
                    way_tag['key'] = child.attrib['k']
                    way_tag['id'] = element.attrib['id']
                    way_tag['value'] = child.attrib['v']
                    tags.append(way_tag)
                    
            elif child.tag == 'nd':
                way_node['id'] = element.attrib['id']
                way_node['node_id'] = child.attrib['ref']
                way_node['position'] = position
                position += 1
                way_nodes.append(way_node)
        
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}




# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.items())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.decode('utf-8') if isinstance(v,bytes) else v) for k, v in row.items()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH, validate=False)

## Create asqlite db and store populates with CSVs##


In [1]:
# Creating the database and tables
import sqlite3
import csv

conn = sqlite3.connect('data_wrangling.sqlite')

conn.text_factory = str
cur = conn.cursor()

#Make some fresh tables using executescript()
cur.execute('''DROP TABLE IF EXISTS nodes''')
cur.execute('''DROP TABLE IF EXISTS nodes_tags''')
cur.execute('''DROP TABLE IF EXISTS ways''')
cur.execute('''DROP TABLE IF EXISTS ways_tags''')
cur.execute('''DROP TABLE IF EXISTS ways_nodes''')


cur.execute('''CREATE TABLE nodes (
    id INTEGER PRIMARY KEY NOT NULL,
    lat REAL,
    lon REAL,
    user TEXT,
    uid INTEGER,
    version INTEGER,
    changeset INTEGER,
    timestamp TEXT)
''')

with open('nodes.csv','r') as nodes_table: # `with` statement available in 2.5+
    # csv.DictReader uses first line in file for column headings by default
    dr = csv.DictReader(nodes_table) # comma is default delimiter
    to_db = [(i['id'], i['lat'], i['lon'], i['user'], i['uid'], i['version'], i['changeset'], i['timestamp']) for i in dr]

cur.executemany("INSERT INTO nodes VALUES (?, ?, ?, ?, ?, ?, ?, ?);", to_db)

#
cur.execute('''CREATE TABLE nodes_tags (
    id INTEGER,
    key TEXT,
    value TEXT,
    type TEXT,
    FOREIGN KEY (id) REFERENCES nodes(id))
''')

with open('nodes_tags.csv','r') as nodes_tags_table: # `with` statement available in 2.5+
    # csv.DictReader uses first line in file for column headings by default
    dr = csv.DictReader(nodes_tags_table) # comma is default delimiter
    to_db = [(i['id'], i['key'], i['value'], i['type']) for i in dr]

cur.executemany("INSERT INTO nodes_tags VALUES (?, ?, ?, ?);", to_db)

#
cur.execute('''CREATE TABLE ways (
    id INTEGER PRIMARY KEY NOT NULL,
    user TEXT,
    uid INTEGER,
    version TEXT,
    changeset INTEGER,
    timestamp TEXT)
''')

with open('ways.csv','r') as ways_table: # `with` statement available in 2.5+
    # csv.DictReader uses first line in file for column headings by default
    dr = csv.DictReader(ways_table) # comma is default delimiter
    to_db = [(i['id'], i['user'], i['uid'], i['version'], i['changeset'], i['timestamp']) for i in dr]

cur.executemany("INSERT INTO ways VALUES (?, ?, ?, ?, ?, ?);", to_db)  

#
cur.execute('''CREATE TABLE ways_tags (
    id INTEGER NOT NULL,
    key TEXT NOT NULL,
    value TEXT NOT NULL,
    type TEXT,
    FOREIGN KEY (id) REFERENCES ways(id))
''')

with open('ways_tags.csv','r') as ways_tags_table: # `with` statement available in 2.5+
    # csv.DictReader uses first line in file for column headings by default
    dr = csv.DictReader(ways_tags_table) # comma is default delimiter
    to_db = [(i['id'], i['key'], i['value'], i['type']) for i in dr]

cur.executemany("INSERT INTO ways_tags VALUES (?, ?, ?, ?);", to_db)

#
cur.execute('''CREATE TABLE ways_nodes (
    id INTEGER NOT NULL,
    node_id INTEGER NOT NULL,
    position INTEGER NOT NULL,
    FOREIGN KEY (id) REFERENCES ways(id),
    FOREIGN KEY (node_id) REFERENCES nodes(id))
''')

with open('ways_nodes.csv','r') as ways_nodes_table: # `with` statement available in 2.5+
    # csv.DictReader uses first line in file for column headings by default
    dr = csv.DictReader(ways_nodes_table) # comma is default delimiter
    to_db = [(i['id'], i['node_id'], i['position']) for i in dr]

cur.executemany("INSERT INTO ways_nodes VALUES (?, ?, ?);", to_db)

#Save changes
conn.commit()

## Files sizes

In [16]:
import os

print ('The Melbourne.osm file is {} MB'.format(os.path.getsize('Melbourne.osm')/1.0e2))
print ('The nodes.csv file is {} MB'.format(os.path.getsize('nodes.csv')/1.0e2))
print ('The nodes_tags.csv file is {} MB'.format(os.path.getsize('nodes_tags.csv')/1.0e2))
print ('The ways.csv file is {} MB'.format(os.path.getsize('ways.csv')/1.0e2))
print ('The ways_nodes.csv file is {} MB'.format(os.path.getsize('ways_nodes.csv')/1.0e2))
print ('The ways_tags.csv file is {} MB'.format(os.path.getsize('ways_tags.csv')/1.0e2))
print ('The data_wrangling.sqlite file is {} MB'.format(os.path.getsize('data_wrangling.sqlite')/1.0e2))

The Melbourne.osm file is 3406934.84 MB
The nodes.csv file is 953806.71 MB
The nodes_tags.csv file is 409174.61 MB
The ways.csv file is 59557.36 MB
The ways_nodes.csv file is 641591.82 MB
The ways_tags.csv file is 401646.6 MB
The data_wrangling.sqlite file is 2225152.0 MB


## Data Overviews

The queries below provides an overview of the Melbourne, AUS OpenStreetMap dataset.

#### The Number of nodes

In [2]:
nodes_count = conn.cursor()
nodes_count.execute("SELECT COUNT(*) FROM nodes")

nodes_count = nodes_count.fetchall()
 
for row in nodes_count:
    print(row)

(2301716,)


#### The Number of ways

In [4]:
ways_count = conn.cursor()
ways_count.execute("SELECT COUNT(*) FROM ways")

ways_count = ways_count.fetchall()
 
for row in ways_count:
    print(row)

(356310,)


#### The top amenities

In [6]:
top_amenities = conn.cursor()
t = ('amenity',)
top_amenities.execute("SELECT value, COUNT(*) as num FROM nodes_tags WHERE key=? GROUP BY value ORDER BY num DESC LIMIT 10", t)

top_amenities = top_amenities.fetchall()
 
for row in top_amenities:
    print(row)

('bench', 2577)
('restaurant', 2076)
('cafe', 1726)
('waste_basket', 1469)
('bicycle_parking', 1427)
('fast_food', 1140)
('post_box', 1012)
('drinking_water', 960)
('toilets', 718)
('telephone', 647)


#### The Places of Worship in the area (extra idea)

In [17]:
warship_area = conn.cursor()

query='''select count(value) from nodes_tags where key = 'amenity' AND value LIKE '%worship%'; '''

result=cur.execute(query)
for row in result:
    print (row)

(377,)


#### The umber of Schoolsextra idea)

In [23]:
query='''select count(value) from nodes_tags where key = 'amenity' AND value LIKE '%School%'; '''

result=cur.execute(query)
for row in result:
    print (row)

(78,)


#### The top sports

In [7]:
top_sports = conn.cursor()
t = ('sport',)
top_sports.execute("SELECT value, COUNT(*) as num FROM nodes_tags WHERE key=? GROUP BY value ORDER BY num DESC LIMIT 10", t)

top_sports = top_sports.fetchall()
 
for row in top_sports:
    print(row)

('basketball', 30)
('fitness', 23)
('swimming', 19)
('cricket_nets', 12)
('skateboard', 12)
('australian_football', 8)
('tennis', 8)
('soccer', 7)
('yoga', 7)
('cricket', 6)


#### The top tourism

In [10]:
top_tourism = conn.cursor()
t = ('tourism', 'tourism_1')
top_tourism.execute("SELECT value, COUNT(*) as num FROM nodes_tags WHERE key=? GROUP BY value UNION SELECT value, COUNT(*) as num FROM nodes_tags WHERE key=? GROUP BY value ORDER BY num DESC", t)

top_tourism = top_tourism.fetchall()
 
for row in top_tourism:
    print(row)

('information', 225)
('picnic_site', 220)
('artwork', 213)
('hotel', 122)
('viewpoint', 103)
('attraction', 65)
('hostel', 32)
('museum', 29)
('motel', 28)
('gallery', 26)
('guest_house', 18)
('apartment', 5)
('caravan_site', 3)
('aquarium', 2)
('camp_site', 2)
('attraction', 1)
('bed_and_breakfast', 1)
('lookout', 1)
('theme_park', 1)


#### The top citys

In [11]:
top_city = conn.cursor()
t = ('city',)
top_city.execute("SELECT value, COUNT(*) as num FROM nodes_tags WHERE key=? GROUP BY value ORDER BY num DESC LIMIT 10", t)

top_city = top_city.fetchall()
 
for row in top_city:
    print(row)

('Glen Waverley', 13307)
('Wantirna South', 7270)
('Wheelers Hill', 6797)
('Mount Waverley', 4226)
('Montmorency', 3773)
('Melbourne', 2802)
('Wantirna', 2746)
('Mulgrave', 1901)
('Vermont South', 1677)
('Greensborough', 1646)


#### The top way of payments

In [12]:
top_payment = conn.cursor()
t = ('payment',)
top_payment.execute("SELECT key, COUNT(*) as num FROM nodes_tags WHERE type=? GROUP BY key ORDER BY num DESC", t)

top_payment = top_payment.fetchall()
 
for row in top_payment:
    print(row)

('coins', 98)
('mastercard', 87)
('visa', 87)
('credit_cards', 55)
('cash', 49)
('telephone_cards', 47)
('bitcoin', 43)
('notes', 39)
('debit_cards', 37)
('american_express', 22)
('contactless', 12)
('maestro', 8)
('cards', 3)
('laser', 3)
('visa_debit', 3)
('litecoin', 2)
('card', 1)
('darkcoin', 1)
('direct_deposit', 1)
('money_order', 1)
('paypal', 1)
('vertcoin', 1)
('visa_electron', 1)


## Conclusion

The OpenStreetMap data is a great place for users to contribute to the data. However, if the integrity and completenenss of data are not maintained, it can't compete with other map services such as Google Maps or Apple Maps which has a better user experience and more structure ways of searching for info.

With more care taken, OpenStreetMap is capable of being the Wikipedia of maps

## Suggestion 

A data validation rules may be helpufl when the data is being entered. The validation could contribute in inproving and  keeping the data consistent by minimizing errors. 
Below query shows all ameneties and how we can bucket some of the sub categories into one:

In [33]:
query='''select DISTINCT value from nodes_tags where key = 'amenity' and value like '%_%'; '''

result=cur.execute(query)
for row in result:
    print (row)

('hospital',)
('place_of_worship',)
('parking_space',)
('toilets',)
('police',)
('pub',)
('fuel',)
('cafe',)
('parking',)
('cinema',)
('public_building',)
('post_box',)
('restaurant',)
('school',)
('shelter',)
('courthouse',)
('bus_station',)
('parking_entrance',)
('fast_food',)
('telephone',)
('boat_rental',)
('theatre',)
('bank',)
('library',)
('post_office',)
('fire_station',)
('drinking_water',)
('bbq',)
('pharmacy',)
('nightclub',)
('kindergarten',)
('doctors',)
('community_centre',)
('atm',)
('bench',)
('childcare',)
('rsl',)
('car_wash',)
('bicycle_parking',)
('fountain',)
('weighbridge',)
('taxi',)
('bar',)
('car_rental',)
('ferry_terminal',)
('recycling',)
('community_hall',)
('veterinary',)
('exercise',)
('clinic',)
('jetty',)
('waste_basket',)
('college',)
('dentist',)
('brothel',)
('ice_cream',)
('university',)
('money_transfer',)
('bureau_de_change',)
('arts_centre',)
('embassy',)
('internet_cafe',)
('restaurant; cafe',)
('stripclub',)
('marketplace',)
('bicycle_rental',)


For in stance, for the name of amenities, there are some obiviously errors in nanes such as:
    - loading_dock
    - driving_school
The "_" should've be allowed into the names of amenities.